In [5]:
import numpy as np
np.random.seed(0)
import os
import sys
import sklearn
sys.path.insert(1, f"{os.getcwd()}/backend")


import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
from matplotlib.ticker import NullFormatter
import seaborn as sns
sns.set(palette="bright",style="ticks",font="Arial")
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import PathPatch
from matplotlib.ticker import FormatStrFormatter

from sklearn import linear_model
from sklearn.linear_model import HuberRegressor
from sklearn import manifold, datasets
from sklearn.decomposition import PCA
from sklearn import preprocessing

from functools import partial
import cvxpy as cp
import pandas as pd
import copy
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics.pairwise import rbf_kernel
import os.path
import pdb
import scipy as sp
import hashlib
import joblib
import pickle
import pdb

from np_backend.dro_conformal import dro_conformal_quantile_procedure_cvx

%load_ext autoreload
%autoreload 2

%env PYTHONHASHSEED=0

env: PYTHONHASHSEED=0


In [ ]:
print(f"numpy: {np.__version__}")
print(f"sys (standard library, version not applicable)")
print(f"matplotlib: {matplotlib.__version__}")
print(f"seaborn: {sns.__version__}")
print(f"sklearn: {sklearn.__version__}")
print(f"cvxpy: {cp.__version__}")
print(f"pandas: {pd.__version__}")
print(f"scipy: {sp.__version__}")
print(f"joblib: {joblib.__version__}")
print(f"pickle (standard library, version not applicable)")

# Settings

In [ ]:
# NOTE: don't forget to set these v.

num_reps = 20
dataset = "bos"
fix_train = True
base_models = ["rf"] # ["lm", "rf", "rob"]
stdize = False

alphas = [0.05]
# alphas = [0.05, 0.1, 0.2, 0.3, 0.4]
alphas.sort()

if(dataset == "gaus"):
    rhos = [0.01]
    calib_test_shift_pcts = [0, 1]

elif(dataset == "airfoil"):
    rhos = [1e-2]
    calib_test_shift_pcts = [0, 1]
    
elif(dataset == "abalone"):
    rhos = [1e-2]
    calib_test_shift_pcts = [0, 1]
    
elif(dataset == "ca"):
    rhos = [1e-2]
    calib_test_shift_pcts = [0, 1]

elif(dataset == "delta"):
    rhos = [1e-2]
    calib_test_shift_pcts = [0, 1]
     
elif(dataset == "ailerons"):
    rhos = [1e-2]
    calib_test_shift_pcts = [0, 1]
    
elif(dataset == "bank"):
    rhos = [1e-2]
    calib_test_shift_pcts = [0, 1]
    
elif(dataset == "bos"):
    rhos = [1e-2]
    calib_test_shift_pcts = [0, 1]
    
elif(dataset == "cpu"):
    rhos = [1e-2]
    calib_test_shift_pcts = [0, 1]
    
elif(dataset == "kin"):
    rhos = [1e-2]
    calib_test_shift_pcts = [0, 1]
    
elif(dataset == "puma"):
    rhos = [1e-2]
    calib_test_shift_pcts = [0, 1]

else:
    print("ERROR: Bad dataset.")
    assert(False)
    
rhos.sort()

kl = lambda z : -cp.entr(z)                     # This is the K-L-ball (it's equivalent to z*cp.log(z),
                                                # just written in a way that's DCP-compliant.
chisq = lambda z : 0.5 * cp.sum_squares(z-1)    # This is the chi-squared ball.

ff= "../data/jasa_10_07_2023_data/"

# Generate data

In [ ]:
if(dataset == "gaus"):
    
    my_n = 5000
    my_p = 10
    
    A = np.random.randn(my_p,my_p)
    A = A.T @ A
    S,Q = np.linalg.eigh(A)
    
    theta0 = Q[:,0]
    theta1 = Q[:,1]
    
    X = np.random.randn(my_n,my_p)
    eps = np.random.randn(my_n)
    y = X @ theta0 + eps
    
    X_shifted = 2 + np.copy(X)
    y_shifted = X_shifted @ theta0 + eps
    
    ts = np.arange(0, 1.1, 0.1)
        
    np.savetxt(ff+dataset + "_X.csv", X, delimiter=",")
    np.savetxt(ff+dataset + "_y.csv", y, delimiter=",")        
        
    np.savetxt(ff+dataset + "_X_shifted.csv", X_shifted, delimiter=",")
    np.savetxt(ff+dataset + "_y_shifted.csv", y_shifted, delimiter=",")         
        
elif(dataset == "airfoil"):

    X = pd.read_csv(ff+dataset + "_X.csv", index_col=0).to_numpy()
    y = pd.read_csv(ff+dataset + "_y.csv", index_col=0).to_numpy()[:,0]
    
elif((dataset == "abalone") or
     (dataset == "ca") or
     (dataset == "delta") or
     (dataset == "ailerons") or
     (dataset == "bank") or
     (dataset == "bos") or
     (dataset == "cpu") or
     (dataset == "kin") or
     (dataset == "puma")):
    
    X = pd.read_csv(ff+dataset + "_X.csv").to_numpy()
    y = pd.read_csv(ff+dataset + "_y.csv").to_numpy()[:,0]

In [ ]:
if stdize:
    X = preprocessing.scale(X)

In [ ]:
n = X.shape[0]
p = X.shape[1]

In [ ]:
alg_names_ordered = ["Standard", "K-L", "Chi-squared"]
alg_idxes = {"Standard":0, "K-L":1, "Chi-squared":2}

shift_taus = [0.5, 0.6, 0.7]

shift_fns = ["I-A", "I-B"]

for dim_idx in range(p):
    shift_fns += ["I-C-" + str(dim_idx)]

for shift_tau in shift_taus:
    shift_fns += ["I-D-" + str(shift_tau)]

for dim_idx in range(p):
    for shift_tau in shift_taus:
        shift_fns += ["I-E-" + str(dim_idx) + "-" + str(shift_tau)]

shift_betas = np.array([0.02, 0.04, 0.08, 0.16, 0.32, 0.64])
shift_betas = np.concatenate([shift_betas, -shift_betas])

In [ ]:
num_algs = len(alg_idxes)
num_mods = len(base_models)
num_alphas = len(alphas)
num_rhos = len(rhos)
num_shift_fns = len(shift_fns)
num_shift_betas = len(shift_betas)

In [ ]:
ws = "  "

In [ ]:
ext_jlib = "jlib"
ext_pkl = "pkl"

In [ ]:
def shift(X, calib_idxes, test_idxes, calib_test_shift_pcts, shift_fn, shift_beta, v, base_qtys):
    
    n_test = len(test_idxes)
    X_test_idxes = X[test_idxes,:]
    X_test_idxes_demeaned = X_test_idxes - np.mean(X_test_idxes, axis=0)
    X_test_idxes_dot_v = base_qtys["X_test_idxes_dot_v"]
    X_test_idxes_demeaned_dot_v = base_qtys["X_test_idxes_demeaned_dot_v"]
    
    X_calib_idxes = X[calib_idxes,:]
    X_calib_idxes_dot_v = base_qtys["X_calib_idxes_dot_v"]
    
    if shift_fn == "I-A":
        w = np.ones(n_test)
    
    elif shift_fn == "I-B":
        w = shift_beta * X_test_idxes_demeaned_dot_v
    
    elif "I-C" in shift_fn:
        dim_idx = int(shift_fn.replace("I-C-", ""))
        w = shift_beta * X_test_idxes_demeaned[:,dim_idx]
    
    elif "I-D" in shift_fn:
        shift_tau = float(shift_fn.replace("I-D-", ""))
        tau = np.quantile(X_calib_idxes_dot_v, shift_tau)
        indicators = np.array(X_test_idxes_dot_v >= tau, dtype="int")
        w = shift_beta * indicators
    
    elif "I-E" in shift_fn:
        dim_idx_shift_tau_str = shift_fn.replace("I-E-", "")
        dash_idx = dim_idx_shift_tau_str.index("-")

        dim_idx_str = dim_idx_shift_tau_str[0:dash_idx]
        shift_tau = float(dim_idx_shift_tau_str.replace(dim_idx_str + "-", ""))
        dim_idx = int(dim_idx_str)
        
        tau = np.quantile(X_calib_idxes[:,dim_idx], shift_tau)
        indicators = np.array(X_test_idxes[:,dim_idx] >= tau, dtype="int")
        w = shift_beta * indicators
        
    if "I-A" not in shift_fn:
        w = np.exp(w - np.max(w))
    
    w = w/np.sum(w)
    
    assert(all(w >= 0))
    assert(all(w <= 1))
    assert(all(np.logical_not(np.isnan(w))))
    assert(all(np.logical_not(np.isinf(w))))
    
    return w

In [ ]:
def compute_coverage_and_length(lo, hi, y, test_idxes, test_idxes_weights):
    
    n_test = len(test_idxes)
    raw_coverages = np.asarray([1. if ((lo[i] <= y[test_idxes[i]]) & (hi[i] >= y[test_idxes[i]]))
                                else 0 for i in range(n_test)])
    coverages = np.multiply(raw_coverages, test_idxes_weights)

    length = np.mean(hi - lo)
    
    return np.sum(coverages), length, raw_coverages

In [ ]:
def update_alpha(alpha_in, n_in):
    return np.maximum(1. - (n_in+1.)*(1.-alpha_in)/n_in, 0.0)

In [ ]:
wksp_fp = ff + dataset + "_wksp." + ext_pkl
test_idxes_weights_wksp_fp = ff + dataset + "_test_idxes_weights_wksp." + ext_jlib

In [ ]:
if os.path.isfile(wksp_fp):
    
    wksp = pickle.load(open(wksp_fp, "rb")) # joblib.load(wksp_fp)
    joblib.load(test_idxes_weights_wksp_fp) # test_idxes_weights_wksp = pickle.load(open(test_idxes_weights_wksp_fp, "rb"))

    train_idxes_list = wksp["train_idxes"]
    base_model_objs_list_dict = wksp["base_model_objs"]

    calib_idxes_list = wksp["calib_idxes"]

    test_idxes_list = wksp["test_idxes"]
    test_idxes_weights_list_dict = test_idxes_weights_wksp["test_idxes_weights"]
    v_list = wksp["v"]
    
    coverages_dict = wksp["coverages_dict"]
    raw_coverages_dict = wksp["raw_coverages_dict"]
    lengths_dict = wksp["lengths_dict"]
    
    wksp_fp_exists = True
    
    if dataset == "gaus":
        ts = wksp["ts"]
    
else:
    
    wksp = {}
    test_idxes_weights_wksp = {}
    
    train_idxes_list = []
    base_model_objs_list_dict = []

    calib_idxes_list = []

    test_idxes_list = []
    test_idxes_weights_list_dict = []
    v_list = []

    if dataset == "gaus":
        lengths_dict = np.inf*np.ones((num_alphas,num_rhos,num_reps,len(ts),num_algs))
        coverages_dict = np.inf*np.ones((num_alphas,num_rhos,num_reps,len(ts),num_algs))

    else:        
        coverages_dict = {}
        raw_coverages_dict = {}
        lengths_dict = np.inf*np.ones((num_alphas,num_rhos,num_reps,num_mods,num_algs))
        for shift_fn_idx, shift_fn in enumerate(shift_fns):

            for shift_beta_idx, shift_beta in enumerate(shift_betas):
                if (shift_fn_idx == 0) and (shift_beta_idx > 0):
                    continue

                coverages_dict[shift_fn, shift_beta] = np.inf*np.ones((num_alphas,num_rhos,num_reps,num_mods,num_algs))
    
    wksp_fp_exists = False

In [ ]:
for rep_idx in range(num_reps):
    print("Trial %d/%d (dataset %s):" % (rep_idx, num_reps - 1, dataset))
    base_qtys = {}
    
    # 1a) If data already exists (i.e. has already been split up +
    # shifted), then load it ... except for the test weights, which
    # we will load later on
    if wksp_fp_exists:
        
        train_idxes = train_idxes_list[rep_idx]
        n_train = len(train_idxes)        
        base_model_objs = base_model_objs_list_dict[rep_idx]
        
        calib_idxes = calib_idxes_list[rep_idx]
        n_calib = len(calib_idxes)
        
        test_idxes = test_idxes_list[rep_idx]
        n_test = len(test_idxes)
        test_idxes_weights_dict = test_idxes_weights_list_dict[rep_idx]
        v = v_list[rep_idx]
        
    # 1b) Else we are seeing data for the first time
    else:
        
        # So, split up the data + save it, first

        # Training data:
        if ((fix_train and (rep_idx == 0)) or not fix_train):
            n_train = int(n / 3.)
            train_idxes = np.random.choice(range(n), n_train, replace=False)
        train_idxes_list += [train_idxes]
            
        # Calibration data:
        n_calib_test = n - n_train
        calib_test_idxes = np.setdiff1d(range(n), train_idxes)
        n_calib = int(n_calib_test / 2.)
        calib_idxes = np.random.choice(calib_test_idxes, n_calib, replace=False)
        calib_idxes_list += [calib_idxes]
        
        # Test data:
        test_idxes = np.setdiff1d(calib_test_idxes, calib_idxes)
        n_test = len(test_idxes)
        test_idxes_list += [test_idxes]
        
        if dataset == "gaus":
            base_qtys["X_test_idxes_dot_v"] = None
            base_qtys["X_test_idxes_demeaned_dot_v"] = None
            base_qtys["X_calib_idxes_dot_v"] = None
        
        else:
            calib_test_idxes = np.concatenate([calib_idxes, test_idxes]).astype(int)
            X_calib_test_idxes = X[calib_test_idxes,:]
            X_calib_test_idxes -= np.mean(X_calib_test_idxes, axis=0)
            _, _, VT = sp.linalg.svd(X_calib_test_idxes)
            v = VT[0,:]
            v_list += [v]

            X_test_idxes = X[test_idxes,:]
            X_calib_idxes = X[calib_idxes,:]        
            base_qtys["X_test_idxes_dot_v"] = X_test_idxes.dot(v)
            base_qtys["X_test_idxes_demeaned_dot_v"] = (X_test_idxes - np.mean(X_test_idxes, axis=0)).dot(v)
            base_qtys["X_calib_idxes_dot_v"] = X_calib_idxes.dot(v)

            test_idxes_weights_dict = {}
            for shift_fn_idx, shift_fn in enumerate(shift_fns):

                for shift_beta_idx, shift_beta in enumerate(shift_betas):
                    if (shift_fn_idx == 0) and (shift_beta_idx > 0):
                        continue
                        
                    test_idxes_weights = shift(X,
                                               calib_idxes,
                                               test_idxes,
                                               calib_test_shift_pcts,
                                               shift_fn,
                                               shift_beta,
                                               v,
                                               base_qtys)
                    test_idxes_weights_dict[shift_fn, shift_beta] = test_idxes_weights
            test_idxes_weights_list_dict += [test_idxes_weights_dict]
            print()

        # b) Fit the base estimator(s) to the training data
        if ((fix_train and (rep_idx == 0)) or not fix_train):
            base_model_objs_dict = {}

            for mod_idx, base_model in enumerate(base_models):

                if(base_model == "lm"):
                    model_obj = linear_model.LinearRegression()
                elif(base_model == "rf"):
                    model_obj = RandomForestRegressor(
                        random_state=0)
                elif(base_model == "rob"):
                    model_obj = HuberRegressor()
                else:
                    print(ws + "ERROR: Bad base_model.")
                    assert(False)
                model_obj.fit(X[train_idxes, :], y[train_idxes])        
                base_model_objs_dict[base_model] = model_obj        
        base_model_objs_list_dict += [base_model_objs_dict]
                    
    # 1c) Report some stats
    print(ws + "Just FYI, here are the various data set sizes:")
    print(ws*2 + "Train: %d." % n_train)
    print(ws*2 + "Calibration: %d." % n_calib)
    print(ws*2 + "Test: %d." % n_test)
    print()
    
    # 2) Compute residuals and scores
    if dataset == "gaus":
                
        alpha_idx = 0
        alpha = alphas[alpha_idx]
        
        w = np.ones(n_test)
        test_idxes_weights = w / np.sum(w)
        
        for t_idx, t in enumerate(ts):
            print(ws*2 + "Processing t=" + str(t) + ".")
            thetat = np.sqrt(1-t**2)*theta0 + t*theta1
            muhat = X @ thetat
            S = (X @ thetat - y)**2

            for rho_idx, rho in enumerate(rhos):

                for alg in alg_names_ordered:
                    print(ws*3 + "Computing " + alg + " intervals.")

                    alg_idx = alg_idxes[alg]
                    if alg == "Standard":
                        q = np.quantile(S, 1.0-update_alpha(alpha, n_calib))

                    elif alg == "K-L":
                        q, _ = dro_conformal_quantile_procedure_cvx(S, kl, update_alpha(
                            alpha, n_calib), rho, want_bisection=True, verbose=False)

                    elif alg == "Chi-squared":
                        q, _ = dro_conformal_quantile_procedure_cvx(
                            S, chisq, update_alpha(alpha, n_calib), rho, want_bisection=True)

                    q = np.sqrt(q)
                    hi = np.add(muhat[test_idxes], q)
                    lo = np.subtract(muhat[test_idxes], q)

                    coverage, length, raw_coverages = compute_coverage_and_length(
                        lo, hi, y_shifted, test_idxes, test_idxes_weights)

                    coverages_dict[alpha_idx,
                                   rho_idx,
                                   rep_idx,
                                   t_idx,
                                   alg_idx] = coverage
                                               
                    lengths_dict[alpha_idx,
                                 rho_idx,
                                 rep_idx,
                                 t_idx,
                                 alg_idx] = length                
                        
    else:    

        for mod_idx, base_model in enumerate(base_models):        

            model_obj = base_model_objs_list_dict[rep_idx][base_model]
            muhat = model_obj.predict(X)            
            S = np.abs(y[calib_idxes] - muhat[calib_idxes])

            # 3) Go thru all possible mis-coverage levels, rho's, conformalization procedures
            for alpha_idx, alpha in enumerate(alphas):

                for rho_idx, rho in enumerate(rhos):

                    for alg in alg_names_ordered:
                        print(ws*4 + "Computing " + alg + " intervals.")

                        alg_idx = alg_idxes[alg]
                        if alg == "Standard":
                            q = np.quantile(S, 1.0-update_alpha(alpha, n_calib))

                        elif alg == "K-L":
                            q, _ = dro_conformal_quantile_procedure_cvx(S, kl, update_alpha(
                                alpha, n_calib), rho, want_bisection=True, verbose=False)

                        elif alg == "Chi-squared":
                            q, _ = dro_conformal_quantile_procedure_cvx(
                                S, chisq, update_alpha(alpha, n_calib), rho, want_bisection=True)

                        hi = np.add(muhat[test_idxes], q)
                        lo = np.subtract(muhat[test_idxes], q)

                        # Evaluate (weighted) coverage
                        for shift_fn_idx, shift_fn in enumerate(shift_fns):

                            for shift_beta_idx, shift_beta in enumerate(shift_betas):
                                if (shift_fn_idx == 0) and (shift_beta_idx > 0):
                                    continue

                                test_idxes_weights = test_idxes_weights_list_dict[rep_idx][shift_fn, shift_beta]
                                coverage, length, raw_coverages = compute_coverage_and_length(
                                    lo, hi, y, test_idxes, test_idxes_weights)

                                coverages_dict[shift_fn, shift_beta][alpha_idx,
                                                                     rho_idx,
                                                                     rep_idx,
                                                                     mod_idx,
                                                                     alg_idx] = coverage

                                raw_coverages_dict[shift_fn, shift_beta, rep_idx, alg_idx] = raw_coverages

                                if shift_fn_idx == 0:                                
                                    lengths_dict[alpha_idx,
                                                 rho_idx,
                                                 rep_idx,
                                                 mod_idx,
                                                 alg_idx] = length

    # 5) Write out coverages and lengths on the last trial
    print()
    if(rep_idx == (num_reps-1)):
        
        print("Finalizing workspace object ...")
        
        if dataset == "gaus":
            wksp["num_reps"] = num_reps
            wksp["dataset"] = dataset
            wksp["fix_train"] = fix_train
            wksp["base_models"] = None
            wksp["stdize"] = stdize
            wksp["alphas"] = alphas
            wksp["rhos"] = rhos
            wksp["shift_fns"] = shift_fns
            wksp["shift_betas"] = shift_betas

            wksp["train_idxes"] = train_idxes_list
            wksp["base_model_objs"] = None

            wksp["calib_idxes"] = calib_idxes_list

            wksp["test_idxes"] = test_idxes_list
            test_idxes_weights_wksp["test_idxes_weights"] = None
            wksp["v"] = None

            wksp["coverages_dict"] = coverages_dict
            wksp["raw_coverages_dict"] = None
            wksp["lengths_dict"] = lengths_dict
            
            wksp["ts"] = ts
        
        else:        
            wksp["num_reps"] = num_reps
            wksp["dataset"] = dataset
            wksp["fix_train"] = fix_train
            wksp["base_models"] = base_models
            wksp["stdize"] = stdize
            wksp["alphas"] = alphas
            wksp["rhos"] = rhos
            wksp["shift_fns"] = shift_fns
            wksp["shift_betas"] = shift_betas

            wksp["train_idxes"] = train_idxes_list
            wksp["base_model_objs"] = base_model_objs_list_dict

            wksp["calib_idxes"] = calib_idxes_list

            wksp["test_idxes"] = test_idxes_list
            test_idxes_weights_wksp["test_idxes_weights"] = test_idxes_weights_list_dict
            wksp["v"] = v_list

            wksp["coverages_dict"] = coverages_dict
            wksp["raw_coverages_dict"] = raw_coverages_dict
            wksp["lengths_dict"] = lengths_dict

In [ ]:
if not wksp_fp_exists:
    pickle.dump(wksp, open(wksp_fp, "wb"))
    joblib.dump(test_idxes_weights_wksp, test_idxes_weights_wksp_fp)
    print("Saved workspaces to %s and %s." % (wksp_fp, test_idxes_weights_wksp_fp))

In [ ]:
print("All done.")